In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv

In [2]:
click_data = pd.read_csv("../data/click_data.csv")
emotional_events = pd.read_csv("../data/emotional_events.csv")
messages_data = pd.read_csv("../data/messages_data.csv")
user_information = pd.read_csv("../data/user_information.csv")
new_features = pd.read_csv("../feature_Engineering/new_features.csv")

### From message_data we are able to extract the number of interactions with the assistand and how long an user takes at a task

In [7]:
messages_data.sort_values(by=["userId", "task", "timestamp_unix"], inplace=True)
messages_data.reset_index(drop=True, inplace=True)

messages_data["number_interactions_with_assistant"] = 0
messages_data["spent_time_with_assistant (seconds)"] = 0

for (userId, task), group in messages_data.groupby(['userId', 'task']):
    number_interactions = group.shape[0]
    
    first_interaction_time = group['timestamp_unix'].min()
    last_interaction_time = group['timestamp_unix'].max()
    
    total_time_spent = last_interaction_time - first_interaction_time
    
    messages_data.loc[group.index, 'number_interactions_with_assistant'] = number_interactions
    messages_data.loc[group.index, 'spent_time_with_assistant (seconds)'] = total_time_spent

    with open('../feature_Engineering/new_features.csv', 'a') as file:
        file.write(f"{userId},{task},{number_interactions},{total_time_spent}\n")


### Exploring click_data, able to extract interactions at a task

In [8]:
click_data.sort_values(by=["userId", "task"], inplace=True)
click_data.reset_index(drop=True, inplace=True)

click_data["number_interactions_with_tables"] = 0
click_data["spent_time_with_tables (seconds)"] = 0

for (userId, task), group in click_data.groupby(['userId', 'task']):
    number_interactions = group.shape[0]
    
    first_interaction_time = group['timestamp_unix'].min()
    last_interaction_time = group['timestamp_unix'].max()
    
    total_time_spent = last_interaction_time - first_interaction_time
    
    click_data.loc[group.index, 'number_interactions_with_tables'] = number_interactions
    click_data.loc[group.index, 'spent_time_with_tables (seconds)'] = total_time_spent
    
    rows = []
    csv_filename = '../feature_Engineering/new_features.csv'
    # Read existing data from the new_features file
    with open(csv_filename, 'r') as file:
        reader = csv.DictReader(file)
        rows = list(reader)

    # Check if the row exists and update it
    row_exists = False
    for row in rows:
        user_feature = int(row['userId'])
        task_feature = int(row['task'])
        userID = int(userId)
        task = int(task)
        if user_feature == userId and task_feature == task:
            row['number_interactions_with_tables'] = number_interactions
            row['spent_time_with_tables (seconds)'] = total_time_spent
            row_exists = True
            break

    # If the row doesn't exist, append a new row
    if not row_exists:
        new_row = {
            'userId': userId,
            'task': task,
            'number_interactions_with_assistant': 0,
            'spent_time_with_assistant (seconds)': 0,
            'number_interactions_with_tables': number_interactions,
            'spent_time_with_tables (seconds)': total_time_spent
        }
        rows.append(new_row)

    # Write the updated data back to the CSV file after each iteration
    with open(csv_filename, 'w', newline='') as file:
        fieldnames = ['userId', 'task', 'number_interactions_with_assistant', 'spent_time_with_assistant (seconds)',
                      'number_interactions_with_tables', 'spent_time_with_tables (seconds)']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)              
    
    